# 🚀 FastAPI Concepts - Greek Derby RAG Chatbot

## Learning Objectives
By the end of this lesson, you will understand:
- What FastAPI is and why it's used
- How to create RESTful APIs with FastAPI
- Request/Response models with Pydantic
- CORS middleware and its importance
- Error handling in FastAPI
- API documentation and testing

---

## Q1: What is FastAPI and why did we choose it for our Greek Derby RAG Chatbot?

**Answer:**

FastAPI is a modern, fast (high-performance) web framework for building APIs with Python 3.7+ based on standard Python type hints. We chose FastAPI for our Greek Derby RAG Chatbot because:

### Key Advantages:
1. **High Performance**: One of the fastest Python frameworks available, comparable to NodeJS and Go
2. **Automatic Documentation**: Generates interactive API docs (Swagger UI) automatically
3. **Type Safety**: Built on Python type hints for better code quality and IDE support
4. **Easy to Use**: Intuitive API design that's easy to learn and use
5. **Standards Based**: Based on OpenAPI and JSON Schema standards
6. **Async Support**: Native support for async/await for better concurrency

### Why Perfect for Our RAG Chatbot:
- **Real-time Communication**: FastAPI handles concurrent requests efficiently
- **API Documentation**: Frontend developers can easily understand our endpoints
- **Type Validation**: Ensures data integrity for chat requests/responses
- **Easy Integration**: Simple to integrate with our React frontend

```python
# Example: Simple FastAPI app
from fastapi import FastAPI

app = FastAPI(title="My API", version="1.0.0")

@app.get("/")
async def read_root():
    return {"message": "Hello World"}
```


## Q2: How do we define API endpoints in our Greek Derby API?

**Answer:**

In our `greek_derby_api.py`, we define endpoints using FastAPI's decorator pattern. Here's how it works:

### Basic Endpoint Structure:
```python
@app.get("/endpoint-name")
async def function_name():
    return response_data
```

### Our API Endpoints:

1. **Root Endpoint** (`GET /`):
   ```python
   @app.get("/")
   async def root():
       return {"message": "🇬🇷 Greek Derby RAG Chatbot API"}
   ```

2. **Health Check** (`GET /health`):
   ```python
   @app.get("/health")
   async def health_check():
       return {"status": "healthy"}
   ```

3. **Chat Endpoint** (`POST /chat`):
   ```python
   @app.post("/chat", response_model=ChatResponse)
   async def chat(request: ChatRequest):
       # Process chat request
       return ChatResponse(answer=response)
   ```

### HTTP Methods Used:
- **GET**: Retrieve data (health check, history, stats)
- **POST**: Send data (chat messages, clear memory)

### Why This Pattern?
- **RESTful Design**: Follows REST principles for clean API design
- **Type Safety**: Each endpoint has defined input/output types
- **Documentation**: FastAPI automatically generates docs from these definitions


## Q3: What are Pydantic models and how do we use them in our API?

**Answer:**

Pydantic models are data validation classes that use Python type annotations to validate data. In our Greek Derby API, we use them to define the structure of our requests and responses.

### Our Pydantic Models:

1. **ChatRequest Model**:
   ```python
   class ChatRequest(BaseModel):
       question: str
   ```
   - Defines the structure of incoming chat requests
   - Validates that `question` is a string
   - Automatically generates JSON schema for documentation

2. **ChatResponse Model**:
   ```python
   class ChatResponse(BaseModel):
       answer: str
       timestamp: str
       conversation_id: Optional[str] = None
   ```
   - Defines the structure of chat responses
   - Ensures consistent response format
   - `Optional[str]` means the field can be None

3. **ConversationHistory Model**:
   ```python
   class ConversationHistory(BaseModel):
       history: List[Dict[str, str]]
       total_messages: int
   ```
   - Complex nested structure for conversation data
   - `List[Dict[str, str]]` represents a list of dictionaries

### Benefits of Pydantic Models:
- **Automatic Validation**: Invalid data is rejected with clear error messages
- **Type Safety**: IDE can provide autocomplete and type checking
- **Documentation**: Automatic generation of API documentation
- **Serialization**: Easy conversion to/from JSON

### Example Usage:
```python
# This will automatically validate the request
@app.post("/chat", response_model=ChatResponse)
async def chat(request: ChatRequest):
    # request.question is guaranteed to be a string
    answer = chatbot.chat(request.question)
    return ChatResponse(
        answer=answer,
        timestamp=datetime.now().isoformat()
    )
```


## Q4: What is CORS and why do we need it in our chatbot API?

**Answer:**

CORS (Cross-Origin Resource Sharing) is a security feature implemented by web browsers that blocks requests from one domain to another domain, unless the server explicitly allows it.

### The Problem:
When our React frontend (running on `http://localhost:3000`) tries to make API calls to our FastAPI backend (running on `http://localhost:8000`), the browser blocks these requests because they're from different origins.

### Our CORS Configuration:
```python
from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow all origins (for development)
    allow_credentials=True,
    allow_methods=["*"],  # Allow all HTTP methods
    allow_headers=["*"],  # Allow all headers
)
```

### What Each Setting Does:
- **`allow_origins=["*"]`**: Allows requests from any domain (development only)
- **`allow_credentials=True`**: Allows cookies and authentication headers
- **`allow_methods=["*"]`**: Allows GET, POST, PUT, DELETE, etc.
- **`allow_headers=["*"]`**: Allows any request headers

### Production Considerations:
In production, you should be more specific:
```python
allow_origins=[
    "https://yourdomain.com",
    "https://www.yourdomain.com"
]
```

### Why We Need It:
Without CORS, our React frontend couldn't communicate with our FastAPI backend, making the chatbot unusable in a web browser.


## Q5: How do we handle errors in our FastAPI application?

**Answer:**

Error handling in FastAPI is crucial for providing a good user experience. Our Greek Derby API implements several error handling strategies:

### 1. HTTPException for API Errors:
```python
from fastapi import HTTPException

# When chatbot is not initialized
if not chatbot:
    raise HTTPException(status_code=500, detail="Chatbot not initialized")

# When question is empty
if not request.question.strip():
    raise HTTPException(status_code=400, detail="Question cannot be empty")

# When processing fails
try:
    answer = chatbot.chat(request.question)
except Exception as e:
    raise HTTPException(status_code=500, detail=f"Error processing question: {str(e)}")
```

### 2. HTTP Status Codes We Use:
- **400 Bad Request**: Invalid input (empty question)
- **500 Internal Server Error**: Server-side errors (chatbot not initialized, processing errors)

### 3. Try-Catch Blocks:
```python
try:
    # Risky operation
    result = some_operation()
except Exception as e:
    # Handle the error gracefully
    raise HTTPException(status_code=500, detail=f"Error: {str(e)}")
```

### 4. Global Error Handling:
FastAPI automatically converts HTTPExceptions to proper HTTP responses:
- Returns appropriate status code
- Includes error message in response body
- Maintains consistent error format

### 5. Error Response Format:
```json
{
    "detail": "Error message here"
}
```

### Why This Approach?
- **User-Friendly**: Clear error messages help users understand what went wrong
- **Debugging**: Detailed error information helps developers fix issues
- **Consistency**: All errors follow the same format
- **Security**: Don't expose sensitive internal information


## Q6: How does FastAPI automatically generate API documentation?

**Answer:**

FastAPI automatically generates interactive API documentation using the OpenAPI standard. This is one of its most powerful features!

### Automatic Documentation Features:

1. **Swagger UI** (available at `/docs`):
   - Interactive documentation interface
   - Test API endpoints directly in the browser
   - See request/response examples
   - Try out different parameters

2. **ReDoc** (available at `/redoc`):
   - Alternative documentation format
   - Clean, readable documentation
   - Better for sharing with team members

### How It Works:

1. **Type Hints**: FastAPI reads your function signatures and type hints
2. **Pydantic Models**: Uses your models to understand data structures
3. **Docstrings**: Reads your function docstrings for descriptions
4. **OpenAPI Schema**: Generates a complete OpenAPI specification

### Example from Our API:
```python
@app.post("/chat", response_model=ChatResponse)
async def chat(request: ChatRequest):
    """Ask a question to the chatbot"""
    # Function implementation
```

This automatically generates:
- Endpoint description: "Ask a question to the chatbot"
- Request schema: Based on `ChatRequest` model
- Response schema: Based on `ChatResponse` model
- Example requests and responses

### Benefits:
- **No Extra Work**: Documentation is always up-to-date
- **Interactive Testing**: Test your API without external tools
- **Team Collaboration**: Easy to share API specifications
- **Client Generation**: Can generate client code in multiple languages

### Accessing Documentation:
- **Swagger UI**: `http://localhost:8000/docs`
- **ReDoc**: `http://localhost:8000/redoc`
- **OpenAPI JSON**: `http://localhost:8000/openapi.json`


## Q7: How do we structure a FastAPI application for scalability?

**Answer:**

Our Greek Derby API follows several best practices for scalable FastAPI applications:

### 1. Application Structure:
```python
# Initialize FastAPI app with metadata
app = FastAPI(
    title="Greek Derby RAG Chatbot API",
    description="API for the Greek Derby (Olympiakos vs Panathinaikos) RAG Chatbot",
    version="1.0.0"
)
```

### 2. Global State Management:
```python
# Global chatbot instance
chatbot = None

@app.on_event("startup")
async def startup_event():
    """Initialize the chatbot when the server starts"""
    global chatbot
    chatbot = GreekDerbyChatbot()
```

### 3. Separation of Concerns:
- **API Layer**: `greek_derby_api.py` - Handles HTTP requests/responses
- **Business Logic**: `greek_derby_chatbot.py` - Contains the RAG logic
- **Models**: Pydantic models for data validation

### 4. Error Handling Strategy:
- Consistent error responses
- Proper HTTP status codes
- Detailed error messages for debugging

### 5. Middleware Configuration:
```python
# CORS middleware for frontend integration
app.add_middleware(CORSMiddleware, ...)
```

### 6. Health Checks:
```python
@app.get("/health")
async def health_check():
    return {
        "status": "healthy",
        "chatbot_loaded": chatbot is not None
    }
```

### 7. Async/Await Pattern:
```python
async def chat(request: ChatRequest):
    # Async function for better performance
    answer = chatbot.chat(request.question)
    return ChatResponse(answer=answer)
```

### Benefits of This Structure:
- **Maintainable**: Clear separation of concerns
- **Testable**: Easy to unit test individual components
- **Scalable**: Can easily add new endpoints and features
- **Documented**: Self-documenting through type hints and docstrings
- **Production-Ready**: Includes health checks and proper error handling


## 🎯 Summary

In this lesson, we learned about the key FastAPI concepts used in our Greek Derby RAG Chatbot:

1. **FastAPI Framework**: Modern, fast, and easy-to-use web framework
2. **API Endpoints**: RESTful design with clear HTTP methods
3. **Pydantic Models**: Type-safe data validation and serialization
4. **CORS Middleware**: Enables cross-origin communication with frontend
5. **Error Handling**: Proper HTTP status codes and error messages
6. **Auto Documentation**: Interactive API docs generated automatically
7. **Scalable Structure**: Clean architecture for maintainable code

### Next Steps:
- Practice creating your own FastAPI endpoints
- Explore the interactive documentation at `/docs`
- Learn about more advanced FastAPI features

### Key Takeaways:
- FastAPI makes API development fast and enjoyable
- Type hints provide better code quality and documentation
- Proper error handling improves user experience
- Automatic documentation saves time and reduces errors

---

**Ready for the next lesson?** We'll explore the RAG (Retrieval-Augmented Generation) concepts used in our chatbot! 🚀
